In [40]:
import csv
RESULTS = [('34 ST-HERALD SQ', 75462.0), ('34 ST-PENN STA', 71010.0), ('TIMES SQ-42 ST', 62494.0), ('GRD CNTRL-42 ST', 44397.0), ('23 ST', 42063.5), ('47-50 STS ROCK', 41853.0), ('59 ST COLUMBUS', 38911.0), ('59 ST', 37528.0), ('86 ST', 34716.0), ('CHAMBERS ST', 34674.0), ('42 ST-BRYANT PK', 31122.0), ('FLUSHING-MAIN', 29207.5), ('50 ST', 28638.0), ('LEXINGTON AV/53', 25864.0), ('ATL AV-BARCLAY', 25023.0), ('42 ST-PORT AUTH', 24921.0), ('JKSN HT-ROOSVLT', 23115.0), ('14 ST', 22358.0), ('JAY ST-METROTEC', 20545.0), ('BOWLING GREEN', 20500.0), ('125 ST', 18639.0), ('BEDFORD AV', 18575.0), ('JAMAICA CENTER', 18368.0), ('WALL ST', 18202.0), ('BOROUGH HALL', 18079.0), ('145 ST', 16984.0), ('49 ST', 16611.5), ('GRAND ST', 15912.0), ('SPRING ST', 14329.0), ('51 ST', 14148.0), ('CHURCH AV', 13451.0), ('CANAL ST', 12710.0), ('1 AV', 12469.0), ('168 ST', 12250.0), ('CROWN HTS-UTICA', 11620.0), ('SUTPHIN-ARCHER', 11532.0), ('JUNCTION BLVD', 10926.0), ('PRINCE ST', 10779.0), ('KINGS HWY', 10528.0), ('BROOKLYN BRIDGE', 10115.0), ('82 ST-JACKSON H', 10079.0), ('DEKALB AV', 10058.0), ('2 AV', 9995.0), ('WORLD TRADE CTR', 9933.0), ('NOSTRAND AV', 9671.0), ('8 ST-NYU', 9560.0), ('28 ST', 9106.0), ('5 AVE', 9099.0), ('RECTOR ST', 8744.0), ('103 ST-CORONA', 8268.0), ('COURT SQ', 8203.0), ('ASTORIA DITMARS', 8175.0), ('MYRTLE-WYCKOFF', 8058.0), ('57 ST-7 AV', 7893.0), ('167 ST', 7851.0), ('FORDHAM RD', 7623.0), ('VERNON-JACKSON', 7575.5), ('161/YANKEE STAD', 7522.0), ('STEINWAY ST', 7230.5), ('LEXINGTON AV/63', 6997.5), ('BLEECKER ST', 6969.0), ('30 AV', 6705.0), ('3 AV-149 ST', 6435.5), ('JAMAICA 179 ST', 6404.0), ('46 ST BLISS ST', 6190.0), ('181 ST', 6043.0), ('EAST BROADWAY', 5941.0), ('NEVINS ST', 5889.0), ('34 ST-HUDSON YD', 5855.0), ('BRIGHTON BEACH', 5762.0), ('BROADWAY', 5688.5), ('63 DR-REGO PARK', 5686.5), ('GRAND-NEWTOWN', 5671.0), ('HOYT-SCHER', 5586.0), ('90 ST-ELMHURST', 5563.0), ('72 ST', 5544.5), ('ASTORIA BLVD', 5444.0), ('96 ST', 5392.0), ('175 ST', 5313.0), ('33 ST-RAWSON ST', 5298.0), ('QUEENSBORO PLZ', 5285.0), ('110 ST', 5173.0), ('40 ST LOWERY ST', 5169.0), ('111 ST', 5096.0), ('HIGH ST', 5089.0), ('ST. GEORGE', 5044.0), ('149/GRAND CONC', 4911.0), ('18 ST', 4785.0), ('GREENPOINT AV', 4740.0), ('135 ST', 4606.5), ('PROSPECT PARK', 4506.0), ('169 ST', 4407.0), ('HOYT ST', 4364.0), ('CENTRAL PK N110', 4351.0), ('191 ST', 4344.0), ('BROADWAY JCT', 4330.0), ('4AV-9 ST', 4308.0), ('21 ST-QNSBRIDGE', 4305.0), ('GRAHAM AV', 4121.0), ('3 AV', 4089.0), ('CARROLL ST', 4057.0), ('OZONE PK LEFFRT', 4019.0), ('NASSAU AV', 3873.0), ('CITY HALL', 3778.0), ('NASSAU ST', 3774.0), ('170 ST', 3760.0), ('FRANKLIN AV', 3684.5), ('ROOSEVELT ISLND', 3622.0), ('BURNSIDE AV', 3588.0), ('18 AV', 3581.0), ('CASTLE HILL AV', 3410.5), ('UNION ST', 3218.0), ('36 AV', 3197.0), ('HALSEY ST', 3120.0), ('COURT SQ-23 ST', 3111.0), ('BEDFORD-NOSTRAN', 3106.0), ('52 ST', 3042.0), ('67 AV', 3036.0), ('PARSONS BLVD', 3036.0), ('WESTCHESTER SQ', 2997.0), ('ELMHURST AV', 2975.0), ('4 AV-9 ST', 2919.0), ('CHRISTOPHER ST', 2884.0), ('SARATOGA AV', 2875.0), ('NEW LOTS AV', 2864.0), ('GRANT AV', 2772.0), ('SUTTER AV-RUTLD', 2761.0), ('PELHAM BAY PARK', 2715.0), ('MORISN AV/SNDVW', 2707.0), ('ROCKAWAY AV', 2687.0), ('EASTN PKWY-MUSM', 2681.0), ('FAR ROCKAWAY', 2681.0), ('SMITH-9 ST', 2681.0), ('MORGAN AV', 2671.0), ('MONTROSE AV', 2666.0), ('CLASSON AV', 2650.0), ('CLARK ST', 2637.0), ('KINGSTON-THROOP', 2617.0), ('PROSPECT AV', 2606.0), ('74 ST-BROADWAY', 2602.0), ('CLINTON-WASH AV', 2600.0), ('DYCKMAN ST', 2575.0), ('BAY RIDGE AV', 2540.0), ('GRAND ARMY PLAZ', 2537.5), ('53 ST', 2486.5), ('FULTON ST', 2464.0), ('V.CORTLANDT PK', 2457.0), ('BAY RIDGE-95 ST', 2366.0), ('231 ST', 2331.0), ('JEFFERSON ST', 2309.0), ('207 ST', 2303.0), ('PARK PLACE', 2243.0), ('EUCLID AV', 2238.0), ('STERLING ST', 2214.0), ('MT EDEN AV', 2156.0), ('BOWERY', 2133.0), ('KINGSTON AV', 2098.0), ('81 ST-MUSEUM', 2096.0), ('PRESIDENT ST', 2085.0), ('AVENUE J', 2080.0), ('15 ST-PROSPECT', 2073.0), ('BEDFORD PK BLVD', 2015.0), ('176 ST', 2005.0), ('WILSON AV', 1990.0), ('183 ST', 1983.5), ('WOODLAWN', 1911.5), ('8 AV', 1905.5), ('45 ST', 1864.0), ('ST LAWRENCE AV', 1829.0), ('BERGEN ST', 1813.0), ('25 ST', 1766.0), ('HARLEM 148 ST', 1737.0), ('77 ST', 1732.0), ('174-175 STS', 1719.0), ('VAN SICLEN AV', 1684.0), ('WOODHAVEN BLVD', 1648.0), ('AVENUE M', 1647.5), ('BRIARWOOD', 1603.0), ('MARBLE HILL-225', 1569.0), ('BEVERLEY ROAD', 1556.0), ('W 8 ST-AQUARIUM', 1535.0), ('CORTELYOU RD', 1519.0), ('138/GRAND CONC', 1516.0), ('71 ST', 1496.5), ('39 AV', 1486.5), ('79 ST', 1459.0), ('CANARSIE-ROCKAW', 1419.0), ('PENNSYLVANIA AV', 1417.0), ('20 AV', 1386.0), ('190 ST', 1360.0), ('METROPOLITAN AV', 1353.0), ('AVENUE H', 1291.5), ('75 ST-ELDERTS', 1251.0), ('OCEAN PKWY', 1235.0), ('182-183 STS', 1218.0), ('85 ST-FOREST PK', 1208.5), ('NECK RD', 1199.0), ('BEACH 67 ST', 1155.0), ('GUN HILL RD', 1151.0), ('BEACH 25 ST', 967.0), ('BEACH 60 ST', 964.0), ('WHITLOCK AV', 919.0), ('AVENUE X', 896.0), ('6 AV', 835.0), ('BEVERLY RD', 813.0), ('BUSHWICK AV', 808.0), ('55 ST', 802.0), ('AQUEDUCT RACETR', 768.0), ('PAVONIA/NEWPORT', 767.0), ('215 ST', 716.5), ('104 ST', 676.0), ('JUNIUS ST', 648.0), ('9TH STREET', 594.5), ('75 AV', 592.0), ('ATLANTIC AV', 583.0), ('CYPRESS HILLS', 531.0), ('BEACH 36 ST', 521.0), ('THIRTY THIRD ST', 508.0), ('GROVE STREET', 480.0), ('AVENUE P', 473.0), ('PATH WTC', 437.5), ('121 ST', 425.0), ('PATH WTC 2', 364.5), ('BEACH 44 ST', 354.0), ('PATH NEW WTC', 337.0), ('NEWARK HM HE', 330.0), ('JOURNAL SQUARE', 271.0), ('EXCHANGE PLACE', 199.5), ('14TH STREET', 199.0), ('LACKAWANNA', 181.0), ('CITY / BUS', 164.0), ('TWENTY THIRD ST', 136.0), ('225 ST', 118.0), ('NEWARK BM BW', 116.0), ('36 ST', 91.0), ('THIRTY ST', 67.0), ('HARRISON', 61.5), ('JFK JAMAICA CT1', 19.0), ('AVENUE I', 13.5), ('NEWARK HW BMEBE', 13.0), ('PARKCHESTER', 12.0), ('137 ST CITY COL', 11.0), ('HUNTS POINT AV', 10.0), ('NEWARK C', 9.0), ('ORCHARD BEACH', 8.0), ('NORWOOD AV', 3.0), ('80 ST', 1.0)]

In [42]:
with open("output.csv",'w') as resultFile:
    wr = csv.writer(resultFile, dialect='excel')
    for item in RESULTS:
        wr.writerow([item,])

In [45]:
df = pd.read_csv("output.csv", names = ["STATION", "DENSIT", "End", "Coverage")
df

TypeError: 'module' object is not subscriptable

In [29]:
locations = []
for key, value in sorted_totals:
    locations.append(key)


In [25]:
import datetime
import time
import pytz 
unixtime = []
ts = df["DATE_TIME"] # have to execute other code first
zone = pytz.timezone('US/Eastern')

In [26]:

dates = '6/29/2018 21:00:00' #example used
date = datetime.datetime.strptime(dates,"%m/%d/%Y %H:%M:%S")
date_eastern= zone.localize(date,is_dst=None)
tmp = date_eastern.astimezone()
print(date_eastern)
epoch3 = date.strftime('%s')
epoch2 = date_eastern.strftime('%s')
epoch = int(time.mktime(tmp.timetuple()))
print(epoch, epoch2,epoch3)


2018-06-29 21:00:00-04:00
1530320400 1530327600 1530331200


In [6]:


#for dt in ts: 
#    dt = entry.to_pydatetime()
#    dt.strptime(dt)
#    if dt.weekday() < 6:
#        tmp = dt.astimezone()
#        epoch = tmp.strftime('%s')
#        unixtime.append(int(epoch))
    
#print(unixtime)
#print(time.tzname)

In [30]:
from yelpapi import YelpAPI
import io, json, pprint
# read API keys (note please make your own config_secret.json)
#{
# "api_key":"YOUR API KEY HERE"
#}
with io.open('config_secret.json') as cred:
    creds = json.load(cred)
limit = 50

def count_dict(dictionary, keyname):
    dict_count = 0
    for item in dictionary.get(keyname):
        dict_count += 1
    return dict_count

def iter_search(offset,limit,loc):
    # make this more flexible in the future
    # take in json file and allow user to set new
    # parameters on the fly
    args = {
    'location': loc,
    'limit': limit,
    'offset': offset,
    'categories': 'coffee,restaurants',
    'open_at': 1530878400,  
    'radius_filter': '241.40160000000003',
    'price': '4'
    }
    search_results = yelp_api.search_query(**args)
    return search_results
    
# input location    
res_dict = {}
yelp_api = YelpAPI(**creds)

for loc in locations: # query the data base 50 offsets at a time
    dict_count = 0
    for offset in range(0,100):
        query_cnt = count_dict(iter_search(offset*limit, limit, loc),'businesses')
        dict_count += query_cnt
        if query_cnt < 50:
            break
           
          
    res_dict[loc] = dict_count  
    
#sorted_rest = sorted(res_dict.items(), key=operator.itemgetter(1), reverse=True)

#pprint.pprint(sorted_rest) 

In [ ]:
#import pandas as pd
#import numpy as np
#import re
#import csv
#locations = []

#csv_input = 'DOITT_SUBWAY_ENTRANCE_01_13SEPT2010.csv'
#df = pd.read_csv(csv_input)
#for loc in df.the_geom:
#    regex = re.compile('\d+.\d+')
#    locations.append(regex.findall(loc))   

In [31]:
res_dict

{'14 ST-UNION SQ': 8,
 '34 ST-HERALD SQ': 25,
 'TIMES SQ-42 ST': 14,
 'FLUSHING-MAIN': 34,
 '59 ST COLUMBUS': 15,
 '34 ST-PENN STA': 12,
 '59 ST': 5,
 'GRD CNTRL-42 ST': 5,
 'JKSN HT-ROOSVLT': 18,
 'W 4 ST-WASH SQ': 7,
 '50 ST': 8,
 '42 ST-PORT AUTH': 14,
 'ATL AV-BARCLAY': 10,
 '145 ST': 21,
 'CHAMBERS ST': 5,
 '8 AV': 10,
 '125 ST': 7,
 '23 ST': 10,
 'BEDFORD AV': 13,
 '47-50 STS ROCK': 1,
 '7 AV': 35,
 '86 ST': 5,
 '103 ST': 13,
 '116 ST': 12,
 '42 ST-BRYANT PK': 14,
 'JAMAICA CENTER': 24,
 'DELANCEY/ESSEX': 9,
 '1 AV': 0,
 "B'WAY-LAFAYETTE": 5,
 'GRAND ST': 9,
 '72 ST-2 AVE': 6,
 'JAY ST-METROTEC': 13,
 '86 ST-2 AVE': 5,
 '49 ST': 15,
 'SUTPHIN-ARCHER': 30,
 'CROWN HTS-UTICA': 15,
 'BOWLING GREEN': 1,
 '82 ST-JACKSON H': 14,
 'FOREST HILLS 71': 35,
 'NOSTRAND AV': 11,
 'BOROUGH HALL': 21,
 '33 ST': 11,
 '66 ST-LINCOLN': 2,
 '161/YANKEE STAD': 36,
 'JUNCTION BLVD': 9,
 'WHITEHALL S-FRY': 6,
 'LEXINGTON AV/53': 15,
 '2 AV': 0,
 '168 ST': 24,
 'DEKALB AV': 27,
 'CANAL ST': 9,
 '167 ST

In [35]:
#for i in range(0,100):
#   print(sorted_totals[i],sorted_rest[i])
new_df = pd.DataFrame()
new_df['locations'] = [x[0] for x in sorted_totals]
new_df['total density'] = [x[1] for x in sorted_totals] 
new_df['proposed number targeted'] = round(0.05*new_df['total density'])
new_df['high priced restraunts'] = [value for key,value in res_dict.items()]
new_df['number people from nearby restraunts'] = ((250/2)*new_df['high priced restraunts'])
#new_df['percentage from restraunts'] = new_df['number people from nearby restraunts']/new_df['total density']
#new_df['proposed number donations'] = new_df['percentage from restraunts']*new_df['proposed number targeted']*0.5

# 250 customers in restraunt at least half come from MTA, at least 50% come from MTA, since nobody wants to drive 


SyntaxError: unexpected EOF while parsing (<ipython-input-35-9f1f1a1ef90c>, line 12)

In [20]:
test = new_df[((new_df['proposed number donations'] > 2.0) & (new_df['ratio of high prices to density'] < 1.0) &  (new_df['proposed number targeted'] > 20))]
test

,locations,total density,proposed number targeted,high priced restraunts,ratio of high prices to density,crime density,proposed number donations
0,YORK ST,1119.0,56.0,6,0.111707,0.2,3.0
1,14 ST-UNION SQ,1113.0,56.0,8,0.112309,0.2,3.0
2,MYRTLE AV,920.0,46.0,28,0.135870,0.2,3.0
3,BEDFORD AV,891.5,45.0,13,0.140213,0.2,3.0
4,GRAND ST,880.0,44.0,9,0.142045,0.2,3.0
5,HOYT-SCHER,858.0,43.0,12,0.145688,0.2,3.0
6,72 ST,836.0,42.0,8,0.149522,0.2,3.0
7,8 AV,825.0,41.0,10,0.151515,0.2,3.0
8,191 ST,817.5,41.0,35,0.152905,0.2,3.0
9,WESTCHESTER SQ,809.0,40.0,46,0.154512,0.2,3.0


In [ ]:
df.loc[(df['real_exits'] < 0) | (df['real_exits'] > 100000), 'real_exits'] = np.nan

In [14]:
df.sort_values('real_exits', ascending=False )

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,real_entries,real_exits,totals,DATE_TIME,time_hour,WEEKDAY
157052,R249,R179,01-00-05,86 ST,456,IRT,07/20/2017,16:00:00,REGULAR,855676885,2097116148,8.522828e+08,2.088503e+09,NaN,2017-07-20 16:00:00,16,3
71286,N215,R237,00-00-02,182-183 STS,BD,IND,06/24/2018,16:00:00,REGULAR,1493118270,1845550240,1.482155e+09,1.838306e+09,NaN,2018-06-24 16:00:00,16,6
41724,N006A,R280,00-00-00,190 ST,A,IND,06/10/2018,20:00:00,REGULAR,1442830408,1845433321,1.431688e+09,1.835852e+09,NaN,2018-06-10 20:00:00,20,6
74342,N305,R017,01-00-04,LEXINGTON AV/53,EM6,IND,06/29/2018,16:00:00,REGULAR,1282912405,1706222667,1.282912e+09,1.706223e+09,NaN,2018-06-29 16:00:00,16,4
151534,R243,R049,00-03-04,51 ST,6,IRT,06/11/2016,20:00:00,REGULAR,1409337658,1610664974,1.403818e+09,1.604317e+09,NaN,2016-06-11 20:00:00,20,5
74353,N305A,R016,00-05-00,LEXINGTON AV/53,EM6,IND,07/08/2017,16:00:00,REGULAR,1090866487,1543778589,1.090860e+09,1.543768e+09,NaN,2017-07-08 16:00:00,16,5
165198,R311,R053,00-00-00,3 AV-149 ST,25,IRT,06/07/2018,16:00:00,REGULAR,1258306153,1476447248,1.253344e+09,1.470647e+09,NaN,2018-06-07 16:00:00,16,3
50937,N072,R012,05-03-01,34 ST-PENN STA,ACE,IND,08/09/2017,16:00:00,REGULAR,1214792949,1367770228,1.211444e+09,1.366924e+09,NaN,2017-08-09 16:00:00,16,2
104185,N601A,R319,01-03-00,LEXINGTON AV/63,F,IND,08/11/2017,20:00:00,REGULAR,80232,805514700,1.730000e+02,8.053067e+08,NaN,2017-08-11 20:00:00,20,4
138517,R204,R043,02-06-00,WALL ST,45,IRT,08/13/2016,18:00:00,REGULAR,184351501,788598212,1.336956e+08,7.709189e+08,NaN,2016-08-13 18:00:00,18,5
